# 4 Generating the Bank for the MicroTasks

To generate the bank for the microtasks I will use an API for an LLM.
The output will be two questions for each core course of each programme.

Basically I will create the perfect prompt that will use the columns of the df_courses to generate the microstasks. 

We use two prompts: broad + disambiguaition (Kenneth Style)

In [1]:
from pathlib import Path
import pandas as pd
#!pip install --upgrade openai
import os, re
from openai import OpenAI
import json
import numpy as np
from tqdm import tqdm  # optional progress bar, pip install tqdm


## 1 Load the data and filter for max 2 courses

In [2]:
# load the csv file about the courses forwhih we have to gen the tasks
silver = Path("../data_programmes_courses/silver")

df_courses_tasks = pd.read_csv(silver / "df_courses_tasks_silver.csv", encoding="utf-8-sig")
print("The shape of the courses tasks dataframe is:", df_courses_tasks.shape)

# keep only first two courses from each programme
df_courses_tasks = df_courses_tasks.groupby("programme_title").head(2).reset_index(drop=True)
print("After keeping only first two courses from each programme the shape is:", df_courses_tasks.shape)

The shape of the courses tasks dataframe is: (36, 21)
After keeping only first two courses from each programme the shape is: (28, 21)


## 2. Set up OpenAI client 

In [3]:

key_path = Path("../data_bank_microtasks") / "api_key.txt"

# Read the key and strip spaces and newlines
api_key = key_path.read_text(encoding="utf8").strip()

# Create the client using this key
client = OpenAI(api_key=api_key)

models = client.models.list()
#for m in models.data:
#    print(m.id)

model_gpt = "gpt-4.1-mini"  


In [4]:
# here we list all programme names
programmes = sorted(df_courses_tasks["programme_title"].unique())
print("Number of programmes:", len(programmes))
print("First few programmes:", programmes[:5])

def build_programme_context(df_prog: pd.DataFrame) -> str:
    """
    With this function we build a short text snippet that describes one programme.
    We use the two core courses that we kept for that programme.
    """
    lines = []

    # we take the programme name from the first row
    prog_name = df_prog["programme_title"].iloc[0]
    lines.append(f"Programme: {prog_name}")

    # we loop over the two core courses
    for idx, row in df_prog.iterrows():
        course_title = row.get("course_name", "")
        course_obj = row.get("course_objective", "")
        course_cont = row.get("course_content", "")

        if isinstance(course_title, str) and course_title.strip():
            lines.append(f"Course: {course_title.strip()}")

        if isinstance(course_obj, str) and course_obj.strip():
            # we keep the course objectives
            lines.append(f"Objectives: {course_obj.strip()}")

        if isinstance(course_cont, str) and course_cont.strip():
            # we keep only a short part of the content to control prompt length
            snippet = course_cont.strip()[:400]
            lines.append(f"Content snippet: {snippet}")

    # we join all lines in a single string
    context = "\n".join(lines)
    return context

# here we test the context builder for one programme
test_prog = programmes[0]
df_test = df_courses_tasks[df_courses_tasks["programme_title"] == test_prog]
print("Context for test programme:")
print(build_programme_context(df_test))


Number of programmes: 14
First few programmes: ['Ancient Studies', 'Biomedical Sciences', 'Business Analytics', 'Communication and Information Studies', 'Computer Science']
Context for test programme:
Programme: Ancient Studies
Course: Objects in Context. An Interdisciplinary Perspective on the Ancient World
Objectives: A distinct feature of Ancient Studies is the combined use of written and material sources. In this course the focus is on material culture, and how this can be used as a source of information about the past. You will familiarize yourself with some of the main categories of objects such as statues, reliefs, coins and utensils as well as with complex assemblages that are part of the sacred, domestic and funerary domain. An important issue is how we evaluate objects as sources of historical information about the past. What are the possibilities and limitations? What types of questions can we ask? For this we take a closer look at the historical context of material sources:

In [5]:
def build_aptitude_prompt(programme_name: str,
                          programme_context: str,
                          task_type: str) -> str:
    """
    With this function we build the text that we send to the model
    to create one aptitude microchallenge.
    task_type can be "classify", "fillblank", "puzzle", "codeorder", or "graph".
    """
    # here we prepare the base instructions that are common for all task types
    base = f"""
You are creating an aptitude micro challenge for a high school student
who is curious about the bachelor programme {programme_name}.

You receive a short context that summarises real courses from this programme.
You must anchor the challenge in that context.
Do not invent random domains.
Stay close to the topics and methods in the context.

The task must test ability or reasoning, not personal preference.
Use instructions such as "Sort these", "Choose the correct", "Complete the text",
"Arrange the code lines", or "Select the correct part of a graph".

General requirements:
- tiny_learn must be a list of exactly three short bullet points.
  Each bullet explains one useful idea in simple language.
- hint must be one short sentence that nudges the student without giving the answer away.
- signalType must always be "aptitude".
"""

    # here we define the type specific instructions
    if task_type == "classify":
        specific = """
Task type: classify.

You must return a JSON object with these fields:
question_code: string, for example "ancient-classify-001"
type: "classify"
signalType: "aptitude"
question: short instruction, for example "Sort these into Greek or Roman origin"
tiny_learn: list of exactly three strings
categories: list of category labels, for example ["Greek", "Roman"]
items: list of objects with fields:
    id: short id such as "a" or "b"
    text: short description of the item
    correctCategory: one of the category labels
hint: short sentence

The categories and items must make sense for this programme.
"""

    elif task_type == "fillblank":
        specific = """
Task type: fillblank.

You must return a JSON object with these fields:
question_code: string, for example "ancient-fillblank-001"
type: "fillblank"
signalType: "aptitude"
question: short instruction, for example "Complete the text"
tiny_learn: list of exactly three strings
textWithBlanks: short text that contains markers {{0}}, {{1}}, etc
blanks: list of objects with fields:
    id: integer index such as 0 or 1
    correctWordId: id of the correct word from the words list
words: list of objects with fields:
    id: string, for example "sumerian"
    text: the word as it should appear in the text
hint: short sentence

The text must describe something that fits the programme context.
"""

    elif task_type == "puzzle":
        specific = """
Task type: puzzle.

You must return a JSON object with these fields:
question_code: string, for example "ancient-puzzle-001"
type: "puzzle"
signalType: "aptitude"
question: short instruction, for example "Which explanation fits best"
tiny_learn: list of exactly three strings
puzzle: object that can include a stem or short description, for example:
    { "variant": "logic", "stem": "..." }
options: list of objects with fields:
    id: short letter id, "A", "B", "C", "D"
    value: the text of the option
correctAnswer: id of the correct option
hint: short sentence

The puzzle must be solvable using the context and standard school knowledge.
"""

    elif task_type == "codeorder":
        specific = """
Task type: codeorder.

You must return a JSON object with these fields:
question_code: string, for example "math-codeorder-001"
type: "codeorder"
signalType: "aptitude"
question: short instruction, for example "Arrange the code lines to compute the correct result"
tiny_learn: list of exactly three strings
language: string naming the language, for example "python"
description: short description of what the code should do in the context of this programme
lines: list of objects with fields:
    id: short id such as "1", "2"
    code: one line of code as a string
    correctPosition: integer that gives the correct position in the final order
expectedOutput: short string that describes what the code prints or returns when correctly ordered
hint: short sentence

The code must be short and readable for a motivated high school student.
It must connect to a context that makes sense for this programme.
"""

    elif task_type == "graph":
        specific = """
Task type: graph.

You must return a JSON object with these fields:
question_code: string, for example "business-graph-001"
type: "graph"
signalType: "aptitude"
question: short instruction, for example "Which part of the graph shows the highest growth"
tiny_learn: list of exactly three strings
graphData: object that describes a simple graph, for example:
    {
      "type": "bar",
      "title": "Exam scores by topic",
      "labels": ["Topic A", "Topic B", "Topic C"],
      "values": [70, 85, 60]
    }
clickableRegions: list of objects with fields:
    id: short id such as "r1", "r2"
    label: the label the student will see for that region or choice
correctRegion: one of the ids from clickableRegions
hint: short sentence

The graph must be simple and should use quantities that match the programme context.
For example, exam scores, sample sizes, counts of artifacts, or similar quantities.
"""

    else:
        raise ValueError(f"Unsupported task_type: {task_type}")

    # here we add the programme context and enforce JSON only output
    context_block = f"""
Programme context:
{programme_context}

Output format:
Return a single valid JSON object and nothing else.
"""

    return base + specific + context_block


In [6]:
import json
import re

def generate_aptitude_task_for_programme(programme_name: str,
                                         programme_context: str,
                                         task_type: str) -> dict:
    """
    With this function we call the model one time and return one aptitude task
    parsed as a Python dict.
    """
    # here we build the full prompt, including programme context and task type
    prompt = build_aptitude_prompt(programme_name, programme_context, task_type)

    # here we call the model
    response = client.responses.create(
        model=model_gpt,
        input=prompt,
        temperature=0.7,
    )

    # here we take the text part of the first output and strip spaces
    raw = response.output[0].content[0].text.strip()

    # small debug print in case something goes wrong
    # we can comment this out later
    print("RAW MODEL OUTPUT START")
    print(raw[:500])
    print("RAW MODEL OUTPUT END")

    # here we try to extract the JSON object from the raw text
    # we look for the first curly brace and the last curly brace
    start = raw.find("{")
    end = raw.rfind("}") + 1

    if start == -1 or end == 0:
        raise ValueError("We did not find any JSON object in the model output")

    json_str = raw[start:end]

    # here we parse the JSON substring into a Python dict
    task = json.loads(json_str)

    # here we enforce signalType and type fields from our side
    task["signalType"] = "aptitude"
    task["type"] = task_type

    # here we normalise tiny_learn to three bullets
    tiny = task.get("tiny_learn", [])
    if not isinstance(tiny, list):
        tiny = [str(tiny)]
    if len(tiny) > 3:
        tiny = tiny[:3]
    while len(tiny) < 3:
        tiny.append("Extra note about the concept.")
    task["tiny_learn"] = tiny

    return task




In [7]:
test_prog = programmes[0]
df_test = df_courses_tasks[df_courses_tasks["programme_title"] == test_prog]
ctx = build_programme_context(df_test)

test_task = generate_aptitude_task_for_programme(
    programme_name=test_prog,
    programme_context=ctx,
    task_type="graph",
)

print(json.dumps(test_task, indent=2, ensure_ascii=False))
print("tiny_learn:", test_task.get("tiny_learn"))
print("Number of bullets:", len(test_task.get("tiny_learn", [])))



RAW MODEL OUTPUT START
```json
{
  "question_code": "ancientstudies-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which category of ancient objects was found most frequently in the Allard Pierson archaeological collection?",
  "tiny_learn": [
    "Material culture includes objects like statues, coins, and utensils that tell us about the past.",
    "Counting objects in different categories helps us understand what was common or important in ancient times.",
    "Graphs can show us which typ
RAW MODEL OUTPUT END
{
  "question_code": "ancientstudies-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which category of ancient objects was found most frequently in the Allard Pierson archaeological collection?",
  "tiny_learn": [
    "Material culture includes objects like statues, coins, and utensils that tell us about the past.",
    "Counting objects in different categories helps us understand what was common or important in ancient times

In [ ]:
from pathlib import Path
import json
from tqdm import tqdm
import random

# here we decide which aptitude task types we want for each programme
# we can change this list later if we add more types
task_types = ["classify", "fillblank", "puzzle", "codeorder", "graph"]

aptitude_bank = {}

# here we loop over all programmes and create aptitude tasks
for prog in tqdm(programmes, desc="Generating aptitude microchallenges"):
    # we filter the two core courses for this programme
    df_prog = df_courses_tasks[df_courses_tasks["programme_title"] == prog]

    # we build the short context snippet for the programme
    ctx = build_programme_context(df_prog)

    tasks_for_prog = []

    # here we generate one task for each type in task_types
    for task_type in task_types:
        try:
            task = generate_aptitude_task_for_programme(
                programme_name=prog,
                programme_context=ctx,
                task_type=task_type,
            )
            tasks_for_prog.append(task)
        except Exception as e:
            # we print the problem and keep going with the next type or programme
            print(f"Problem for programme {prog} task type {task_type}: {e}")

    # we store the aptitude tasks for this programme
    aptitude_bank[prog] = {
        "aptitude": tasks_for_prog
    }

print("Example programme:", programmes[0])
print(json.dumps(aptitude_bank[programmes[0]]["aptitude"], indent=2, ensure_ascii=False))

# ================================================================
# here we merge aptitude with the existing personality bank
# ================================================================

data_dir = Path("../data_bank_microtasks")
data_dir.mkdir(parents=True, exist_ok=True)

# here we load the existing microtasks bank if it exists
base_bank_path = data_dir / "microtasks_bank.json"

if base_bank_path.exists():
    with open(base_bank_path, "r", encoding="utf-8") as f:
        microtasks_bank = json.load(f)
    print("Loaded existing microtasks_bank.json")
else:
    # if we do not have a previous bank we start from an empty dict
    microtasks_bank = {}
    print("No existing microtasks_bank.json found, we start from an empty bank")

# here we merge aptitude tasks into the main bank
for prog, block in aptitude_bank.items():
    # we make sure the programme entry exists in the main bank
    if prog not in microtasks_bank:
        microtasks_bank[prog] = {}
    # we replace or create the aptitude list for this programme
    microtasks_bank[prog]["aptitude"] = block["aptitude"]

# ================================================================
# here we save the full bank
# ================================================================


Generating aptitude microchallenges:   0%|          | 0/14 [00:00<?, ?it/s]

RAW MODEL OUTPUT START
```json
{
  "question_code": "ancient-classify-001",
  "type": "classify",
  "signalType": "aptitude",
  "question": "Classify these ancient items as either material culture or literary source",
  "tiny_learn": [
    "Material culture includes physical objects like statues, coins, and utensils.",
    "Literary sources are written texts like epic poems, historical records, or inscriptions.",
    "Comparing both types helps us understand historical contexts and their reliability."
  ],
  "categori
RAW MODEL OUTPUT END
RAW MODEL OUTPUT START
```json
{
  "question_code": "ancient-fillblank-001",
  "type": "fillblank",
  "signalType": "aptitude",
  "question": "Complete the text by filling in the blanks with the correct words.",
  "tiny_learn": [
    "Material objects like statues and coins help us learn about ancient people and their culture.",
    "Comparing written and material sources can reveal different perspectives on the same historical event.",
    "Knowing t

Generating aptitude microchallenges:   7%|▋         | 1/14 [00:30<06:40, 30.78s/it]

RAW MODEL OUTPUT START
```json
{
  "question_code": "ancientstudies-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which type of ancient object was found most frequently in the domestic archaeological assemblage?",
  "tiny_learn": [
    "Material culture includes objects like statues, coins, and utensils.",
    "Different domains like sacred, domestic, and funerary have distinct object types.",
    "Counting objects helps us understand how people lived and what was important to them."
  ],
 
RAW MODEL OUTPUT END
RAW MODEL OUTPUT START
```json
{
  "question_code": "biomed-genetics-classify-001",
  "type": "classify",
  "signalType": "aptitude",
  "question": "Classify these genetic features as belonging to either prokaryotic or eukaryotic genomes.",
  "tiny_learn": [
    "Prokaryotic genomes are usually circular and lack a nucleus.",
    "Eukaryotic genomes are linear, organized into chromosomes inside a nucleus.",
    "Gene expression processes differ between

Generating aptitude microchallenges:  14%|█▍        | 2/14 [01:00<06:02, 30.19s/it]

...

RAW MODEL OUTPUT START
```json
{
  "question_code": "mkda-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which period in European cultural history had the highest number of significant cultural events according to the graph?",
  "tiny_learn": [
    "Graphs can show how many events happened in different time periods.",
    "Comparing bar heights helps identify the period with the most events.",
    "Understanding historical periods helps contextualize cultural changes."
  ],
  "graphData": {

RAW MODEL OUTPUT END
RAW MODEL OUTPUT START
```json
{
  "question_code": "philosophy-classify-001",
  "type": "classify",
  "signalType": "aptitude",
  "question": "Classify these philosophical topics as belonging to 'Ancient Philosophy' or 'Epistemology'.",
  "tiny_learn": [
    "Ancient Philosophy studies the earliest philosophical traditions and their core questions about reality and knowledge.",
    "Epistemology focuses specifically on the nature and justification of

Generating aptitude microchallenges:  93%|█████████▎| 13/14 [06:28<00:29, 29.82s/it]

RAW MODEL OUTPUT START
```json
{
  "question_code": "philosophy-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which tradition shows the highest number of primary texts analyzed in the Ancient Philosophy course?",
  "tiny_learn": [
    "Ancient Philosophy covers Greco-Roman, Sanskrit, and Chinese traditions.",
    "Primary texts are analyzed to understand philosophical development in each tradition.",
    "Comparing quantities helps identify which tradition is emphasized most in the course."
RAW MODEL OUTPUT END
RAW MODEL OUTPUT START
```json
{
  "question_code": "ppe-ethics-methods-classify-001",
  "type": "classify",
  "signalType": "aptitude",
  "question": "Classify these concepts as belonging to Ethics or Methods of PPE I",
  "tiny_learn": [
    "Ethics studies moral theories and how to apply them to real-world issues.",
    "Methods of PPE I teaches formal reasoning, logic, and mathematical techniques.",
    "Some concepts focus on understanding values,

Generating aptitude microchallenges: 100%|██████████| 14/14 [06:55<00:00, 29.70s/it]

RAW MODEL OUTPUT START
```json
{
  "question_code": "ppe-graph-001",
  "type": "graph",
  "signalType": "aptitude",
  "question": "Which ethical theory has the highest number of students favoring it in the survey?",
  "tiny_learn": [
    "Ethical theories like consequentialism and Kantian ethics offer different ways to evaluate actions.",
    "Graphs can show how preferences or opinions are distributed across groups.",
    "Comparing values in a graph helps identify the most popular or significant category."
  ],
  "g
RAW MODEL OUTPUT END
Example programme: Ancient Studies
[
  {
    "question_code": "ancient-classify-001",
    "type": "classify",
    "signalType": "aptitude",
    "question": "Classify these ancient items as either material culture or literary source",
    "tiny_learn": [
      "Material culture includes physical objects like statues, coins, and utensils.",
      "Literary sources are written texts like epic poems, historical records, or inscriptions.",
      "Comparing

In [11]:

full_bank_path = data_dir / "microchallenges_bank_aptitude.json"

with open(full_bank_path, "w", encoding="utf-8") as f:
    json.dump(microtasks_bank, f, ensure_ascii=False, indent=2)

print("Saved full microtasks bank to:", full_bank_path)
print("Number of programmes in the full bank:", len(microtasks_bank))

# here we quickly check that one programme has both personality and aptitude if personality existed
sample_prog = programmes[0]
print("Sample programme:", sample_prog)
print("Keys for this programme:", microtasks_bank.get(sample_prog, {}).keys())


Saved full microtasks bank to: ..\data_bank_microtasks\microchallenges_bank_aptitude.json
Number of programmes in the full bank: 14
Sample programme: Ancient Studies
Keys for this programme: dict_keys(['aptitude'])


In [4]:
from pathlib import Path
import json

# here we set the directory where we store all banks
data_dir = Path("../data_bank_microtasks")
data_dir.mkdir(parents=True, exist_ok=True)

# here we define the paths of the two source jsons
personality_path = data_dir / "microtasks_RIASEC.json"
aptitude_path = data_dir / "microchallenges_bank_aptitude.json"

# here we load the personality bank
with open(personality_path, "r", encoding="utf-8") as f:
    personality_bank = json.load(f)

print("We loaded the personality bank with programmes:",
      len(personality_bank))

# here we load the aptitude bank
with open(aptitude_path, "r", encoding="utf-8") as f:
    aptitude_bank = json.load(f)

print("We loaded the aptitude bank with programmes:",
      len(aptitude_bank))

# here we start from the personality bank as base
full_bank = personality_bank.copy()

# here we merge aptitude blocks into the full bank
for prog, block in aptitude_bank.items():
    # we make sure there is a dict for this programme
    if prog not in full_bank:
        full_bank[prog] = {}
    # we take the aptitude list from the aptitude bank
    full_bank[prog]["aptitude"] = block.get("aptitude", [])

# here we save the full merged bank
full_bank_path = data_dir / "microtasks_bank_full.json"

with open(full_bank_path, "w", encoding="utf-8") as f:
    json.dump(full_bank, f, ensure_ascii=False, indent=2)

print("We saved the full bank to:", full_bank_path)

# here we quickly inspect one programme to see the keys
sample_prog = "Ancient Studies"
if sample_prog in full_bank:
    print("Sample programme:", sample_prog)
    print("Blocks for this programme:", list(full_bank[sample_prog].keys()))
else:
    print("Warning, Ancient Studies is not in the merged bank")


We loaded the personality bank with programmes: 14
We loaded the aptitude bank with programmes: 14
We saved the full bank to: ..\data_bank_microtasks\microtasks_bank_full.json
Sample programme: Ancient Studies
Blocks for this programme: ['broad', 'R', 'I', 'A', 'S', 'E', 'C', 'aptitude']
